### implemtning simple agetic AI rag for learning

### in this project agentic RAg will check the result set in local 
### if the result set not found in local it will try to fetch the webserach from api and get the relevant information

Agentic RAG Architecture Steps:
	1.	User QueryThe user submits a query to the system.
	2.	Router Agent DecisionAn agent analyzes the query to decide if the answer can be found in the local knowledge base or if an external search is necessary.
	3.	Retrieval
	•	If local: Relevant chunks are retrieved from the document knowledge base using semantic search.
	•	If not local: Agents perform a web search and scrape/extract relevant information.
	4.	Context CompilationResults from retrieval (local or web) are consolidated as the supporting context.
	5.	LLM GenerationThe context plus the original query are sent to a large language model (LLM) which generates the final response.
	6.	Response DeliveryThe answer is provided to the user in natural language.

In [1]:
import openai
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import qdrant_client
from qdrant_client.models import PointStruct

/Users/nageshmuniraja/AI_Basics_25/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from dotenv import load_dotenv
import os
# Load environment variables from .env
load_dotenv()

# Read your API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
#PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT") 

In [2]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
client = qdrant_client.QdrantClient("http://localhost:6333")

In [3]:
def get_webpage_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()

In [5]:
def split_text(text, chunk_size=150):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]


In [6]:
def generate_embeddings(chunks):
    return embedding_model.encode(chunks)

In [7]:
def store_embeddings_db(embeddings, chunks):
    points = [
        PointStruct(id=i, vector=emb.tolist(), payload={"text": chunk})
        for i, (emb, chunk) in enumerate(zip(embeddings, chunks))
    ]
    client.upsert(collection_name="documents", points=points)

In [8]:
def search_vector_db(query, top_k=3):
    q_embedding = embedding_model.encode([query])[0]
    return client.search(collection_name="documents", query_vector=q_embedding.tolist(), top=top_k)

In [9]:
def generate_response(query, context):
    prompt = f"Answer the following using the context: {query}\n\nContext: {context}"
    response = openai.Completion.create(
        engine="text-davinci-003", prompt=prompt, max_tokens=150
    )
    return response.choices[0].text.strip()

In [11]:
def perform_online_search(query):
    # Replace with your API key
    search_url = f"https://www.googleapis.com/customsearch/v1?q={query}&key=YOUR_API_KEY"
    results = requests.get(search_url).json()
    return " ".join([item["snippet"] for item in results.get("items", [])])

In [12]:
def agentic_rag_system(query, url):
    content = get_webpage_content(url)
    chunks = split_text(content)
    embeddings = generate_embeddings(chunks)
    store_embeddings_db(embeddings, chunks)
    search_results = search_vector_db(query)
    if search_results:
        context = "\n".join([res.payload["text"] for res in search_results])
        return generate_response(query, context)
    else:
        online_context = perform_online_search(query)
        return generate_response(query, online_context)

In [13]:
query = "What is Llama 3?"
url = "https://example.com/llama3-article"
print(agentic_rag_system(query, url))

ResponseHandlingException: [Errno 61] Connection refused